In [1]:
# Packaging Import 
import numpy as np 
import pandas as pd 
import statsmodels.regression.linear_model as lm 
import statsmodels.tools.tools as ct
import datetime as dt 
import statsmodels.api as sm 
import pandas_datareader.data as reader
import pandas_datareader.famafrench as ff
import yfinance as yf 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network  import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.model_selection import train_test_split
#Fama French 3 Factor Model Data
yf.pdr_override()
end = dt.date.today()
y = int(input("Enter the no of years: "))
start = dt.date(end.year -y,end.month,end.day)
portfolio = input("Enter the Company : ")
retd = reader.get_data_yahoo(portfolio, start=start, end=end )['Adj Close'].pct_change()
retm  = retd.resample( 'M').agg(lambda x:(x+1).prod() - 1)
retm = retm[1:((y*12) - 1)]

factors = pd.read_csv('/Users/dvparikh/Downloads/F-F_Research_Data_5_Factors_2x3 2.csv')



Enter the no of years: 3
Enter the Company : COROMANDEL.NS
[*********************100%***********************]  1 of 1 completed


In [2]:
factors['Date'] = factors['Date'].astype(str)

# Convert the 'Date' column to the datetime format
factors['Date'] = pd.to_datetime(factors['Date'], format='%Y%m')

# Convert the 'Date' column to the desired date format (optional)
factors['Date'] = factors['Date'].dt.strftime('%Y-%m-%d')

In [3]:
factors['Date'] = pd.to_datetime(factors['Date'], format='%Y-%m-%d')

In [4]:
retm.index.dtype


dtype('<M8[ns]')

In [5]:
factors

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-01,-0.39,-0.41,-0.97,0.68,-1.18,0.27
1,1963-08-01,5.07,-0.80,1.80,0.36,-0.35,0.25
2,1963-09-01,-1.57,-0.52,0.13,-0.71,0.29,0.27
3,1963-10-01,2.53,-1.39,-0.10,2.80,-2.01,0.29
4,1963-11-01,-0.85,-0.88,1.75,-0.51,2.24,0.27
...,...,...,...,...,...,...,...
713,2022-12-01,-6.41,-0.16,1.32,0.09,4.19,0.33
714,2023-01-01,6.65,4.43,-4.05,-2.62,-4.53,0.35
715,2023-02-01,-2.58,0.69,-0.78,0.90,-1.41,0.34
716,2023-03-01,2.51,-7.01,-9.01,1.92,-2.29,0.36


In [6]:

factors.dtypes

Date      datetime64[ns]
Mkt-RF           float64
SMB              float64
HML              float64
RMW              float64
CMA              float64
RF               float64
dtype: object

In [7]:
start = pd.to_datetime(start)
end = pd.to_datetime(end)

In [8]:
select_data = factors[(factors['Date'] >= start) & (factors['Date'] <= end)]

In [9]:
select_data
select_data = select_data.set_index('Date')

In [10]:
select_data

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2020-07-01,5.77,-3.20,-1.37,0.40,0.96,0.01
2020-08-01,7.63,-0.89,-2.96,4.26,-1.20,0.01
2020-09-01,-3.63,0.01,-2.68,-1.39,-1.89,0.01
2020-10-01,-2.10,4.64,4.22,-0.75,-0.76,0.01
2020-11-01,12.47,7.12,2.13,-2.20,1.37,0.01
2020-12-01,4.63,4.79,-1.50,-1.99,-0.11,0.01
2021-01-01,-0.03,6.92,2.97,-3.81,4.90,0.01
2021-02-01,2.78,4.53,7.17,0.31,-1.94,0.00
2021-03-01,3.08,-0.86,7.39,6.47,3.41,0.00


In [11]:
retm


Date
2020-07-31    0.048853
2020-08-31   -0.025702
2020-09-30    0.016841
2020-10-31   -0.059908
2020-11-30    0.140114
2020-12-31   -0.016599
2021-01-31    0.037196
2021-02-28   -0.079883
2021-03-31    0.004149
2021-04-30   -0.042153
2021-05-31    0.079997
2021-06-30    0.124368
2021-07-31    0.025888
2021-08-31   -0.136274
2021-09-30    0.001955
2021-10-31   -0.011140
2021-11-30   -0.057790
2021-12-31    0.021413
2022-01-31    0.044177
2022-02-28   -0.012194
2022-03-31    0.033208
2022-04-30    0.118747
2022-05-31    0.056285
2022-06-30    0.008731
2022-07-31    0.085941
2022-08-31    0.023670
2022-09-30   -0.055930
2022-10-31   -0.032036
2022-11-30   -0.033875
2022-12-31   -0.045819
2023-01-31    0.008905
2023-02-28    0.010543
2023-03-31   -0.021479
2023-04-30    0.071879
Freq: M, Name: Adj Close, dtype: float64

In [12]:
retm.shape


(34,)

In [13]:
select_data.shape

(34, 6)

In [14]:
retm.index = select_data.index

In [15]:
retm

Date
2020-07-01    0.048853
2020-08-01   -0.025702
2020-09-01    0.016841
2020-10-01   -0.059908
2020-11-01    0.140114
2020-12-01   -0.016599
2021-01-01    0.037196
2021-02-01   -0.079883
2021-03-01    0.004149
2021-04-01   -0.042153
2021-05-01    0.079997
2021-06-01    0.124368
2021-07-01    0.025888
2021-08-01   -0.136274
2021-09-01    0.001955
2021-10-01   -0.011140
2021-11-01   -0.057790
2021-12-01    0.021413
2022-01-01    0.044177
2022-02-01   -0.012194
2022-03-01    0.033208
2022-04-01    0.118747
2022-05-01    0.056285
2022-06-01    0.008731
2022-07-01    0.085941
2022-08-01    0.023670
2022-09-01   -0.055930
2022-10-01   -0.032036
2022-11-01   -0.033875
2022-12-01   -0.045819
2023-01-01    0.008905
2023-02-01    0.010543
2023-03-01   -0.021479
2023-04-01    0.071879
Name: Adj Close, dtype: float64

In [16]:
merge = pd.merge(retm,select_data,on='Date')

In [17]:
merge

,Adj Close,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
2020-07-01,0.048853,5.77,-3.20,-1.37,0.40,0.96,0.01
2020-08-01,-0.025702,7.63,-0.89,-2.96,4.26,-1.20,0.01
2020-09-01,0.016841,-3.63,0.01,-2.68,-1.39,-1.89,0.01
2020-10-01,-0.059908,-2.10,4.64,4.22,-0.75,-0.76,0.01
2020-11-01,0.140114,12.47,7.12,2.13,-2.20,1.37,0.01
2020-12-01,-0.016599,4.63,4.79,-1.50,-1.99,-0.11,0.01
2021-01-01,0.037196,-0.03,6.92,2.97,-3.81,4.90,0.01
2021-02-01,-0.079883,2.78,4.53,7.17,0.31,-1.94,0.00
2021-03-01,0.004149,3.08,-0.86,7.39,6.47,3.41,0.00


In [18]:
merge[['Mkt-RF','SMB','HML','RF','RMW','CMA']]= merge[['Mkt-RF','SMB','HML','RF','RMW','CMA']]/100

In [19]:
merge

,Adj Close,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
2020-07-01,0.048853,0.0577,-0.0320,-0.0137,0.0040,0.0096,0.0001
2020-08-01,-0.025702,0.0763,-0.0089,-0.0296,0.0426,-0.0120,0.0001
2020-09-01,0.016841,-0.0363,0.0001,-0.0268,-0.0139,-0.0189,0.0001
2020-10-01,-0.059908,-0.0210,0.0464,0.0422,-0.0075,-0.0076,0.0001
2020-11-01,0.140114,0.1247,0.0712,0.0213,-0.0220,0.0137,0.0001
2020-12-01,-0.016599,0.0463,0.0479,-0.0150,-0.0199,-0.0011,0.0001
2021-01-01,0.037196,-0.0003,0.0692,0.0297,-0.0381,0.0490,0.0001
2021-02-01,-0.079883,0.0278,0.0453,0.0717,0.0031,-0.0194,0.0000
2021-03-01,0.004149,0.0308,-0.0086,0.0739,0.0647,0.0341,0.0000


In [20]:
merge['Ri-Rf']= merge['Adj Close'] - merge.RF

In [21]:
merge

,Adj Close,Mkt-RF,SMB,HML,RMW,CMA,RF,Ri-Rf
Date,,,,,,,,
2020-07-01,0.048853,0.0577,-0.0320,-0.0137,0.0040,0.0096,0.0001,0.048753
2020-08-01,-0.025702,0.0763,-0.0089,-0.0296,0.0426,-0.0120,0.0001,-0.025802
2020-09-01,0.016841,-0.0363,0.0001,-0.0268,-0.0139,-0.0189,0.0001,0.016741
2020-10-01,-0.059908,-0.0210,0.0464,0.0422,-0.0075,-0.0076,0.0001,-0.060008
2020-11-01,0.140114,0.1247,0.0712,0.0213,-0.0220,0.0137,0.0001,0.140014
2020-12-01,-0.016599,0.0463,0.0479,-0.0150,-0.0199,-0.0011,0.0001,-0.016699
2021-01-01,0.037196,-0.0003,0.0692,0.0297,-0.0381,0.0490,0.0001,0.037096
2021-02-01,-0.079883,0.0278,0.0453,0.0717,0.0031,-0.0194,0.0000,-0.079883
2021-03-01,0.004149,0.0308,-0.0086,0.0739,0.0647,0.0341,0.0000,0.004149


In [22]:
y = merge['Ri-Rf']
x = merge[['Mkt-RF','SMB','HML','RMW','CMA']]
x_sm = sm.add_constant(x)

In [23]:
model= sm.OLS(y,x_sm)
res = model.fit()

In [24]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Ri-Rf   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                 -0.030
Method:                 Least Squares   F-statistic:                    0.8059
Date:                Sun, 11 Jun 2023   Prob (F-statistic):              0.555
Time:                        11:31:06   Log-Likelihood:                 49.956
No. Observations:                  34   AIC:                            -87.91
Df Residuals:                      28   BIC:                            -78.75
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0057      0.012      0.486      0.630      -0.018       0.030
Mkt-RF         0.1586      0.212      0.746      0.462      -0.277       0.594
SMB            0.1979      0.451      0.439      0.664      -0.726       1.122
HML           -0.3872      0.357     -1.085      0.287      -1.118       0.344
RMW           -0.2254      0.459     -0.491      0.627      -1.166       0.716
CMA            0.8493      0.480      1.770      0.088      -0.133       1.832
==============================================================================
Omnibus:                        0.175   Durbin-Watson:                   1.752
Prob(Omnibus):                  0.916   Jarque-Bera (JB):                0.296
Skew:                           0.151   Prob(JB):                        0.862
Kurtosis:                       2.656   Cond. No.                         61.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
model1 = RandomForestRegressor(n_estimators =1000, random_state =0)
model1.fit(x,y)
y_pred1 = model1.predict(x)
r21 = model1.score(x,y)

In [26]:
r21

0.8573396800416337

In [27]:
y_pred1

array([ 0.02514329, -0.01861288, -0.00028117, -0.0482258 ,  0.08976258,
       -0.00721988,  0.0352472 , -0.06353761,  0.00826638, -0.03108121,
        0.05533045,  0.07712915,  0.01057039, -0.09581618,  0.00293719,
       -0.01965712, -0.02955983,  0.01237871,  0.03664483,  0.00082909,
        0.02032056,  0.07934802,  0.04560197,  0.01470588,  0.05134665,
        0.02230855, -0.03083729, -0.00539842, -0.01769021, -0.01792766,
        0.01611096, -0.00222164, -0.00073265,  0.04496719])

In [28]:

# Assuming you have prepared your dataset and named it 'data'
# 'data' should include the Fama-French factors as input features and the dependent variable

# Split the data into input features (X) and the dependent variable (y)
X = merge[['Mkt-RF', 'SMB', 'HML', 'CMA','RMW']]
y = merge['Ri-Rf']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the random forest regression model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

# Calculate the feature importance scores
feature_importance = model.feature_importances_
print("Feature Importance:", feature_importance)



R-squared: -0.08133835557557312
Feature Importance: [0.16888258 0.11073029 0.19467632 0.23873163 0.28697919]
